# Парсер под HH

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pprint


In [15]:
def hh_Parse(vacancy='python'):
    page_num = 1
    base_link='https://hh.ru'
    
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
    header = {
        'User-Agent':user_agent
    }
    vacancies_hh = []

    while True:
        main_link = f'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={vacancy}&page={str(page_num)}'
        response= requests.get(main_link,headers = header).text
        soup = bs(response,'lxml')
        try:
            next_link_part = soup.find_all('a',{'class':'HH-Pager-Controls-Next'})[0]['href']
            next_link = base_link+next_link_part
        except:
            break

        if next_link ==f'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text={vacancy}&page={str(page_num+1)}':

           # response= requests.get(main_link,headers = header).text
            #soup = bs(response,'lxml')
            vacancy_block = soup.find_all('div',{'class':'vacancy-serp'})[0]
            vacancy_all = vacancy_block.find_all('div',{'class':'vacancy-serp-item'
            })
            for vac in vacancy_all:
                vacancy_data = {}
                vacancy_name = vac.find_all('div',{'class':'vacancy-serp-item__info'})[0].find('a').getText()
                vacancy_link = vac.find_all('div',{'class':'vacancy-serp-item__info'})[0].find('a')['href']
                if '-' in vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText():
                    vacancy_money_min, vacancy_money_max = vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText().split('-')
                    vacancy_money_max, currency = vacancy_money_max.rsplit(' ',1)
                    currency = currency.rstrip('.')  
                    vacancy_data['vacancy_money_min'] = vacancy_money_min.replace('\xa0', ' ')
                    vacancy_data['vacancy_money_max'] = vacancy_money_max.replace('\xa0', ' ')
                    vacancy_data['currency'] = currency
                elif '0' in vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText():
                    vacancy_money,currency = vac.find_all('div',{'class':'vacancy-serp-item__sidebar'})[0].getText().rsplit(' ',1)
                    currency = currency.rstrip('.')
                    vacancy_data['vacancy_money'] = vacancy_money.replace('\xa0', ' ')
                    vacancy_data['currency'] = currency
                else:
                    vacancy_money = None
                    vacancy_data['vacancy_money'] = vacancy_money
                company = vac.find_all('div',{'class':'vacancy-serp-item__meta-info'})[0].getText().split('-')[0]
                info = [vac.find_all('div',{'class':'g-user-content'})[e].getText() for e in range(len(vac.find_all('div',{'class':'g-user-content'})))]
                source = main_link
                to_respond = base_link+vac.find_all('div',{'class':'vacancy-serp-item__controls-item vacancy-serp-item__controls-item_response HH-VacancyResponseTrigger-Button'})[0].find('a')['href']
                vacancy_data['vacancy_name'] = vacancy_name
                vacancy_data['vacancy_link'] = vacancy_link
                vacancy_data['company'] = company.replace('\xa0', ' ')
                vacancy_data['info'] = info
                vacancy_data['source'] = source
                vacancy_data['to_respond']=to_respond
                vacancies_hh.append(vacancy_data)
            page_num+=1
            
    return vacancies_hh

In [19]:
hh_Parse('Golang')

[{'vacancy_money': None,
  'vacancy_name': 'Go/C++ developer',
  'vacancy_link': 'https://hh.ru/vacancy/36297018?query=Golang',
  'company': 'ООО Акронис',
  'info': ['Understand and analyze product requirements and communicate with product management teams. Create high-quality code and pass code reviews with...3+ years of experience in software development/engineering. Programming skills in Go and in C++. Experience with MySQL or...'],
  'source': 'https://hh.ru/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=Golang&page=1',
  'to_respond': 'https://hh.ru/applicant/vacancy_response?vacancyId=36297018&hhtmFrom=vacancy_search_result'},
 {'vacancy_money': None,
  'vacancy_name': 'C++ team lead',
  'vacancy_link': 'https://hh.ru/vacancy/36663085?query=Golang',
  'company': 'ООО Акронис',
  'info': ['Дизайн и разработка нового функционала (от анализа требований до презентации сделанной работы). Управление командой от 4 человек. Написание автоматических и

# Парсер Super Job

In [8]:
def sj_parse(vacancy = 'python'):
    page_num = 1
    base_link=f'https://www.superjob.ru'

    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
    header = {
        'User-Agent':user_agent
    }
    vacancies_SJ = []
    count = 1
    while True:

        main_link = f'https://russia.superjob.ru/vacancy/search/?keywords={vacancy}&page={page_num}'
        page_num+=1
        response= requests.get(main_link,headers = header).text
        soup = bs(response,'lxml')
        try:
            vacancy_block = soup.find_all('div',{'class':'_1ID8B'})[0]
        except:
            break
        vacancy_all = vacancy_block.find_all('div',{'class':'QiY08 LvoDO'})

        for vac in vacancy_all:
            vacancy_data = {}
            vacancy_name = vac.find_all('a',{'class':'icMQ_'})[0].getText()
            vacancy_link = base_link+vac.find_all('a',{'class':'icMQ_'})[0]['href']
            if vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})  == []:
                vacancy_money = None
            elif '—' in vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText():
                vacancy_money_min, vacancy_money_max = vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText().split('—')
                vacancy_money_max, currency = vacancy_money_max.replace('\xa0', ' ').rsplit(' ',1)
                currency = currency.rstrip('.')  
                vacancy_data['vacancy_money_min'] = vacancy_money_min.replace('\xa0', ' ')
                vacancy_data['vacancy_money_max'] = vacancy_money_max
                vacancy_data['currency'] = currency
            elif '0' in vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText():
                vacancy_money,currency = vac.find_all('span',{'class':'_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})[0].getText().replace('\xa0', ' ').rsplit(' ',1)
                currency = currency.rstrip('.')
                vacancy_data['vacancy_money'] = vacancy_money.replace('\xa0', ' ')
                vacancy_data['currency'] = currency
            else:
                vacancy_money = None
                vacancy_data['vacancy_money'] = vacancy_money
            try:
                company = vac.find_all('span',{'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'})[0].getText()
            except:
                company = None
            info = [vac.find_all('div',{'class':'_3cLIl _3C76h _10Aay _2_FIo _1tH7S'})[e].getText() for e in range(len(vac.find_all('div',{'class':'_3cLIl _3C76h _10Aay _2_FIo _1tH7S'})))]
            source = main_link
            to_respond = None
            vacancy_data['vacancy_name'] = vacancy_name
            vacancy_data['vacancy_link'] = vacancy_link
            vacancy_data['company'] = company
            vacancy_data['info'] = info
            vacancy_data['source'] = source
            vacancy_data['to_respond']=to_respond
            vacancies_SJ.append(vacancy_data)
    return vacancies_SJ

In [20]:
#sj_parse()